In [2]:
# Import ElementTree module for XML parsing
import xml.etree.ElementTree as ET

# Import defaultdict for creating the inverted index and term frequencies
from collections import defaultdict

# Import the math module for mathematical operations
import math

# Import the json module for reading stop words from a file
import json

class SimpleIRSystem:
    # Initialize the SimpleIRSystem class with necessary attributes
    def __init__(self, stop_words_file):
        self.index = defaultdict(list) # Inverted index mapping terms to document IDs
        self.stop_words = self._get_stop_words(stop_words_file) # Set of stop words
        self.doc_term_frequencies = defaultdict(dict) # Term frequencies in documents
        self.doc_lengths = {}  # Length of each document
        self.total_docs = 0 # Total number of documents
        self.avg_doc_length = 0 # Average document length
        self.k1 = 1.5  # BM25 parameter
        self.b = 0.75   # BM25 parameter

    # Retrieve stop words from the provided JSON file
    def _get_stop_words(self, stop_words_file):
        with open(stop_words_file, 'r') as f:
            stop_words = json.load(f)
        return set(stop_words)

    # Check if the character is alphanumeric
    def _is_alphanumeric(self, char):
        return char.isalnum()

    # Preprocess the documents by tokenizing, converting to lowercase, removing alphanumeric characters, filtering stop words
    def _preprocess_text(self, text):
        tokens = text.lower().split()
        processed_tokens = [
            token for token in tokens
            if self._is_alphanumeric(token) and token not in self.stop_words
        ]
        return processed_tokens

    # Index documents from the provided XML file
    def index_documents_from_xml(self, xml_file):
      with open(xml_file, 'r') as file:
        doc_data = file.read()

      start_index = doc_data.find("<doc>")
      while start_index != -1:
        end_index = doc_data.find("</doc>", start_index)
        if end_index == -1:
            break

        doc_xml = doc_data[start_index:end_index + len("</doc>")]
        self._index_single_doc_from_xml(doc_xml)
        start_index = doc_data.find("<doc>", end_index + len("</doc>"))

      self.avg_doc_length = sum(self.doc_lengths.values()) / len(self.doc_lengths)

    # Index a single document from its XML representation
    def _index_single_doc_from_xml(self, doc_xml):
      doc_root = ET.fromstring(doc_xml)
      doc_id_element = doc_root.find('docno')
      text_element = doc_root.find('text')

      if doc_id_element is not None and text_element is not None:
        doc_id = doc_id_element.text.strip()
        text = text_element.text
        if text is not None:
            processed_text = self._preprocess_text(text.strip())
            self._update_index(doc_id, processed_text)
            self.doc_lengths[doc_id] = len(processed_text)
            self.total_docs += 1
        else:
            print("Warning: Empty <text> tag found in doc")
      else:
        print("Warning: <docno> or <text> tag not found in doc")

    # Update the inverted index and document term frequencies
    def _update_index(self, doc_id, processed_text):
      for term in processed_text:
        self.index[term].append(doc_id)

        # Increment term frequency for the document
        if term in self.doc_term_frequencies[doc_id]:
            self.doc_term_frequencies[doc_id][term] += 1
        else:
            self.doc_term_frequencies[doc_id][term] = 1

    # Calculate the query vector for a given query from the query XML file
    def calculate_query_vector(self, query):
        query_vector = defaultdict(float)
        for term in query:
            if term in self.index:
                query_vector[term] += 1
        return query_vector

    # Display the indexed documents
    def display_indexed_documents(self):
      for term, doc_ids in self.index.items():
        print(f"Term : {term}")
        print(f"Document IDs: {doc_ids}")
        print()

    # Calculate cosine similarity between query vector and a document
    def cosine_similarity(self, query_vector, doc_id):
        dot_product = sum(query_vector[term] * self.doc_term_frequencies[doc_id].get(term, 0)
                          for term in query_vector)
        query_vector_length = math.sqrt(sum(val ** 2 for val in query_vector.values()))
        doc_vector_length = math.sqrt(sum(val ** 2 for val in self.doc_term_frequencies[doc_id].values()))
        if query_vector_length == 0 or doc_vector_length == 0:
            return 0
        return dot_product / (query_vector_length * doc_vector_length)

    # Calculate BM25 score for a given document and query  terms
    def bm25_score(self, query_terms, doc_id):
        score = 0
        doc_length = self.doc_lengths.get(doc_id, 0)
        for term in query_terms:
            tf = self.index[term].count(doc_id)
            df = len(self.index[term])
            idf = math.log((self.total_docs - df + 0.5) / (df + 0.5) + 1)
            score += idf * (tf * (self.k1 + 1)) / (tf + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length))
        return score

    # Calculate relevance score
    def _calculate_relevance_score(self, query):
      scores = defaultdict(float)
      query_terms = self._preprocess_text(query)
      for term in query_terms:
        if term in self.index and len(self.index[term]) > 0:  # Add a check for non-zero length
            idf = math.log(len(self.doc_lengths) / len(self.index[term]))
            for doc_id in self.index[term]:
                tf = self.doc_term_frequencies[doc_id].get(term, 0) / self.doc_lengths[doc_id]
                scores[doc_id] += idf * tf
      return scores


    # Rank documents using Vector Space Model
    def rank_documents_vsm(self, query):
        query_vector = self.calculate_query_vector(query)
        scores = defaultdict(float)
        for doc_id in self.doc_lengths:
            scores[doc_id] = self.cosine_similarity(query_vector, doc_id)

        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return ranked_docs

    # Rank documents using BM25 Model
    def rank_documents_bm25(self, query):
        scores = defaultdict(float)
        for doc_id in self.doc_lengths:
            scores[doc_id] = self.bm25_score(query, doc_id)

        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return ranked_docs

    # Rank documents based on relevance score
    def rank_documents_relevance_based(self, query):
        scores = self._calculate_relevance_score(query)
        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return ranked_docs

    # Read queries from XML file
    @staticmethod
    def read_queries_from_xml(xml_file):
        queries = []
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for top in root.findall('top'):
            num = top.find('num').text.strip()
            title = top.find('title').text.strip()
            queries.append((num, title))
        return queries

# Define file paths
stop_words_file = '/content/stopwords-en.json'
xml_file_path = '/content/cran.all.1400.xml'
query_xml_file = '/content/cran.qry.xml'

# Initialize IR system
ir_system = SimpleIRSystem(stop_words_file)

# Index documents
ir_system.index_documents_from_xml(xml_file_path)

# Read queries from XML
queries = ir_system.read_queries_from_xml(query_xml_file)

# Display indexed documents
ir_system.display_indexed_documents()

# Process and rank queries using Vector Space Model
vsm_results = []
for query_num, query_text in queries:
    query_terms = ir_system._preprocess_text(query_text)
    ranked_docs = ir_system.rank_documents_vsm(query_terms)

    for rank, (doc_id, similarity) in enumerate(ranked_docs, start=1):
        result_vsm = f"{query_num} 0 {doc_id} 0 {similarity:.4f} 0"
        vsm_results.append(result_vsm)

# Save Vector Space Model results to a file
with open('vsm_results.txt', 'w') as f:
    f.write('\n'.join(vsm_results))

# Process and rank queries using BM25 Model
bm25_results = []
for query_num, query_text in queries:
    query_terms = ir_system._preprocess_text(query_text)
    ranked_docs = ir_system.rank_documents_bm25(query_terms)

    for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
        result_bm = f"{query_num} 0 {doc_id} 0 {score:.4f} 0"
        bm25_results.append(result_bm)

# Save BM25 Model results to a file
with open('bm25_results.txt', 'w') as f:
    f.write('\n'.join(bm25_results))

# Process and rank queries using Relevance-based Language Model
rlm_results = []
for query_num, query_text in queries:
    ranked_docs = ir_system.rank_documents_relevance_based(query_text)

    for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
        result = f"{query_num} 0 {doc_id} 0 {score:.4f} 0"
        rlm_results.append(result)

# Save Relevance-based Language Model results to a file
with open('rlm_results.txt', 'w') as f:
    f.write('\n'.join(rlm_results))

Streaming output truncated to the last 5000 lines.
Document IDs: ['704', '705', '705', '705', '705', '705', '705', '705', '705', '705', '781', '1280', '1280']

Term : schemes
Document IDs: ['704', '704', '968']

Term : routines
Document IDs: ['704']

Term : processing
Document IDs: ['704', '790']

Term : flapping
Document IDs: ['704']

Term : allmovable
Document IDs: ['704']

Term : separating
Document IDs: ['705', '918', '961', '1385']

Term : nonsingular
Document IDs: ['705']

Term : possio
Document IDs: ['705']

Term : som
Document IDs: ['706']

Term : wagner
Document IDs: ['706', '1330', '1330']

Term : reports
Document IDs: ['706', '1318']

Term : broad
Document IDs: ['706', '883', '892', '1339', '1383']

Term : notable
Document IDs: ['706']

Term : gliders
Document IDs: ['707']

Term : distributes
Document IDs: ['707']

Term : redistribution
Document IDs: ['707', '833']

Term : enhanced
Document IDs: ['707']

Term : reentry
Document IDs: ['708', '708', '709', '709', '711', '711',

In [1]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308216 sha256=d84b1fd9ed325fd13cf0f54dbe2387a2174492206451f5070e7837a979803352
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [4]:
import pytrec_eval

# Load ground truth data
with open('/content/cranqrel.trec.txt', 'r') as f:
    qrels = pytrec_eval.parse_qrel(f)

# Load retrieval results from files
with open('vsm_results.txt', 'r') as f:
    vsm_results = pytrec_eval.parse_run(f)

with open('bm25_results.txt', 'r') as f:
    bm25_results = pytrec_eval.parse_run(f)

with open('rlm_results.txt', 'r') as f:
    rlm_results = pytrec_eval.parse_run(f)

evaluation_measure = ['map', 'P_5', 'ndcg']

evaluator = pytrec_eval.RelevanceEvaluator(qrels, evaluation_measure)

# Evaluate VSM results
vsm_eval = evaluator.evaluate(vsm_results)
vsm_results_map = pytrec_eval.compute_aggregated_measure('map', [query_eval['map'] for query_eval in vsm_eval.values()])
vsm_results_p5 = pytrec_eval.compute_aggregated_measure('P_5', [query_eval['P_5'] for query_eval in vsm_eval.values()])
vsm_results_ndcg = pytrec_eval.compute_aggregated_measure('ndcg', [query_eval['ndcg'] for query_eval in vsm_eval.values()])

print("VSM Evaluation Results:")
print("MAP:", vsm_results_map)
print("P@5:", vsm_results_p5)
print("NDCG:", vsm_results_ndcg)

# Evaluate BM25 results
bm25_eval = evaluator.evaluate(bm25_results)
bm25_results_map = pytrec_eval.compute_aggregated_measure('map', [query_eval['map'] for query_eval in bm25_eval.values()])
bm25_results_p5 = pytrec_eval.compute_aggregated_measure('P_5', [query_eval['P_5'] for query_eval in bm25_eval.values()])
bm25_results_ndcg = pytrec_eval.compute_aggregated_measure('ndcg', [query_eval['ndcg'] for query_eval in bm25_eval.values()])

print("\nBM25 Evaluation Results:")
print("MAP:", bm25_results_map)
print("P@5:", bm25_results_p5)
print("NDCG:", bm25_results_ndcg)

# Evaluate RLM results
rlm_eval = evaluator.evaluate(rlm_results)
rlm_results_map = pytrec_eval.compute_aggregated_measure('map', [query_eval['map'] for query_eval in rlm_eval.values()])
rlm_results_p5 = pytrec_eval.compute_aggregated_measure('P_5', [query_eval['P_5'] for query_eval in rlm_eval.values()])
rlm_results_ndcg = pytrec_eval.compute_aggregated_measure('ndcg', [query_eval['ndcg'] for query_eval in rlm_eval.values()])

print("\nRLM Evaluation Results:")
print("MAP:", rlm_results_map)
print("P@5:", rlm_results_p5)
print("NDCG:", rlm_results_ndcg)


VSM Evaluation Results:
MAP: 0.01761566245593373
P@5: 0.015789473684210527
NDCG: 0.22125514138351435

BM25 Evaluation Results:
MAP: 0.015584595152063136
P@5: 0.01578947368421053
NDCG: 0.21766366322333902

RLM Evaluation Results:
MAP: 0.012016957454062284
P@5: 0.013157894736842105
NDCG: 0.11172845403733751
